In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import os.path as osp
import sys
import models.pose_mobilenet_relu_bnfuse
import torch
import argparse

def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)
        
# this_dir = osp.dirname(__file__)
# lib_path = osp.join(this_dir, '..')
add_path('..')

In [2]:
model = PoseMobileNet()   
debug = True
checkpoint_prefix = 'model/Ultralight-Nano-SimplePose'
_, mxnet_weights, mxnet_aux = mxnet.model.load_checkpoint(checkpoint_prefix, 0)
remapped_state = {}
#print('mxnet_weights:', mxnet_weights) # beta gamma weight
#print('mxnet_aux:', mxnet_aux) # running_mean running_var

'''#print model's state_dict
print('Model.state_dict:')
for n,param_tensor in enumerate(model.state_dict()):
    #打印 key value字典
    print(n, param_tensor,'\t',model.state_dict()[param_tensor].size())'''

"#print model's state_dict\nprint('Model.state_dict:')\nfor n,param_tensor in enumerate(model.state_dict()):\n    #打印 key value字典\n    print(n, param_tensor,'\t',model.state_dict()[param_tensor].size())"

In [ ]:
mxnet_key = 
mxnet_array = mxnet_aux[mxnet_key] if aux else mxnet_weights[mxnet_key]

In [3]:
 for state_key in model.state_dict().keys():
        k = state_key.split('.') # pytorch
        # mobilenetv21_features_conv0_weight
        # mobilenetv21_features_linearbottleneck0_batchnormcudnnoff0_gamma
        # hybridsequential1_batchnormcudnnoff0_gamma
        aux = False
        mxnet_key = ''
        if k[0] == 'features':
            if k[1] == 'conv1_1':
                # input block
                mxnet_key += 'conv1_x_1__'
                if k[2] == 'bn':
                    mxnet_key += 'relu-sp__bn_'
                    aux, key_add = _convert_bn(k[3])
                    mxnet_key += key_add
                else:
                    assert k[3] == 'weight'
                    mxnet_key += 'conv_' + k[3]
            elif k[1] == 'conv5_bn_ac':
                # bn + ac at end of features block
                mxnet_key += 'conv5_x_x__relu-sp__bn_'
                assert k[2] == 'bn'
                aux, key_add = _convert_bn(k[3])
                mxnet_key += key_add
            else:
                # middle blocks
                if model.b and 'c1x1_c' in k[2]:
                    bc_block = True  # b-variant split c-block special treatment
                else:
                    bc_block = False
                ck = k[1].split('_')
                mxnet_key += ck[0] + '_x__' + ck[1] + '_'
                ck = k[2].split('_')
                mxnet_key += ck[0] + '-' + ck[1]
                if ck[1] == 'w' and len(ck) > 2:
                    mxnet_key += '(s/2)' if ck[2] == 's2' else '(s/1)'
                mxnet_key += '__'
                if k[3] == 'bn':
                    mxnet_key += 'bn_' if bc_block else 'bn__bn_'
                    aux, key_add = _convert_bn(k[4])
                    mxnet_key += key_add
                else:
                    ki = 3 if bc_block else 4
                    assert k[ki] == 'weight'
                    mxnet_key += 'conv_' + k[ki]
        elif k[0] == 'classifier':
            if 'fc6-1k_weight' in mxnet_weights:
                mxnet_key += 'fc6-1k_'
            else:
                mxnet_key += 'fc6_'
            mxnet_key += k[1]
        else:
            assert False, 'Unexpected token'
 
        if debug:
            print(mxnet_key, '=> ', state_key, end=' ')
 
        mxnet_array = mxnet_aux[mxnet_key] if aux else mxnet_weights[mxnet_key]
        torch_tensor = torch.from_numpy(mxnet_array.asnumpy())
        if k[0] == 'classifier' and k[1] == 'weight':
            torch_tensor = torch_tensor.view(torch_tensor.size() + (1, 1))
        remapped_state[state_key] = torch_tensor
 
        if debug:
            print(list(torch_tensor.size()), torch_tensor.mean(), torch_tensor.std())

AttributeError: 'PoseMobileNet' object has no attribute 'b'

In [ ]:
model.load_state_dict(remapped_state)